In [5]:
import sys
sys.path.append("..") # Adds higher directory to python modules path
from loaddata import BengaliAI, load_df
from tqdm import tqdm

In [1]:
# df, _ = load_df(True)
# dataset = BengaliAI(df[:10000])
# dataloader = DataLoader(dataset, batch_size=32)